In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
data_gen_train=ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2,horizontal_flip=True)
train_set=data_gen_train.flow_from_directory('Dataset/train',target_size=(64,64),batch_size=32,class_mode='binary')

Found 8000 images belonging to 2 classes.


In [11]:
data_gen_test=ImageDataGenerator(rescale=1./255)
test_set=data_gen_test.flow_from_directory('Dataset/test',batch_size=32,target_size=(64,64),class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [18]:
cnn=Sequential()
cnn.add(Conv2D(filters=32,kernel_size=3,activation='relu',padding="same",input_shape=[64,64,3]))
cnn.add(MaxPool2D(pool_size=2,strides=2))
cnn.add(Conv2D(filters=32,kernel_size=3,activation='relu',padding="same",input_shape=[64,64,3]))
cnn.add(MaxPool2D(pool_size=2,strides=2))
cnn.add(Flatten())
cnn.add(Dense(units=128,kernel_initializer='he_uniform',activation='relu'))
cnn.add(Dense(units=50,kernel_initializer='he_uniform',activation='relu'))
cnn.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [19]:
cnn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [20]:
cnn.fit(x=train_set,validation_data=test_set,epochs=10)

Epoch 1/10
250/250 [==============================] - 593s 2s/step - loss: 0.6452 - accuracy: 0.6202 - val_loss: 0.6263 - val_accuracy: 0.6470
Epoch 2/10
250/250 [==============================] - 719s 3s/step - loss: 0.5775 - accuracy: 0.6957 - val_loss: 0.5549 - val_accuracy: 0.7240
Epoch 3/10
250/250 [==============================] - 676s 3s/step - loss: 0.5364 - accuracy: 0.7286 - val_loss: 0.5198 - val_accuracy: 0.7475
Epoch 4/10
250/250 [==============================] - 722s 3s/step - loss: 0.5159 - accuracy: 0.7464 - val_loss: 0.5563 - val_accuracy: 0.7220
Epoch 5/10
250/250 [==============================] - 710s 3s/step - loss: 0.4867 - accuracy: 0.7635 - val_loss: 0.5356 - val_accuracy: 0.7515
Epoch 6/10
250/250 [==============================] - 579s 2s/step - loss: 0.4639 - accuracy: 0.7816 - val_loss: 0.4865 - val_accuracy: 0.7790
Epoch 7/10
250/250 [==============================] - 411s 2s/step - loss: 0.4526 - accuracy: 0.7849 - val_loss: 0.4652 - val_accuracy: 0.7810

In [21]:
cnn.summary()
 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 50)               

In [22]:
cnn.evaluate(test_set)

63/63 [==============================] - 28s 446ms/step - loss: 0.4553 - accuracy: 0.7920


[0.45526713132858276, 0.7919999957084656]

In [26]:
from tensorflow.keras.models import load_model
cnn.save("Model-dog-cat.h5")

In [31]:
model=load_model("Model-dog-cat.h5")

In [38]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 50)               

## Testing the model by giving different individual inputs

In [77]:
def outcome(folder,name,num):
    from tensorflow.keras.preprocessing import image
    import numpy as np
    test_img=image.load_img(f"Dataset/test/{folder}/{name}.{num}.jpg",target_size=(64,64))
    test_img=image.img_to_array(test_img)
    test_img=test_img/255
    test_img=np.expand_dims(test_img,axis=0)
    result=model.predict(test_img)
    if result>0.5:
         print("The given image is of a dog")
    else:
        print("The given image is of a cat")

In [80]:
outcome("cats","cat","4005")

The given image is of a cat


In [81]:
outcome("dogs","dog","4008")

The given image is of a dog
